In [9]:
import requests
import pandas as pd
from datetime import datetime
import time

def get_TVL():
    try:
        url = "https://api.llama.fi/v2/historicalChainTvl/solana"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
    except:
        return False

    # Conversion en DataFrame
    df_tvl = pd.DataFrame(data)

    # Transformation du timestamp en date
    df_tvl['date'] = pd.to_datetime(df_tvl['date'], unit='s')
    df_tvl.set_index('date', inplace=True)

    # On garde uniquement la colonne TVL en USD
    df_tvl = df_tvl[['tvl']].rename(columns={'tvl': 'TVL_USD'})

    # Optionnel : filtrer les 5 dernières années
    df_tvl = df_tvl[df_tvl.index >= (pd.Timestamp.today() - pd.DateOffset(years=6))]
    df_tvl.to_csv("SOL_tvl.csv")
    return True


def get_funding_rates(symbol="SOLUSDT", interval_hours=8, days=5*365):
    endpoint = "https://fapi.binance.com/fapi/v1/fundingRate"
    start_time = datetime.utcnow() - timedelta(days=days)
    start_ts = int(start_time.timestamp() * 1000)

    all_data = []
    while start_ts < int(datetime.utcnow().timestamp() * 1000):
        params = {
            "symbol": symbol,
            "startTime": start_ts,
            "limit": 1000
        }
        response = requests.get(endpoint, params=params)
        data = response.json()

        if not data:
            break

        all_data.extend(data)

        # Update start time for next batch
        start_ts = data[-1]['fundingTime'] + 1

        # Respect API rate limit
        time.sleep(0.5)

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = pd.to_numeric(df['fundingRate'], errors='coerce')

    # Resample to daily by summing 3 funding rates per day
    df.set_index('fundingTime', inplace=True)
    df_daily = df['fundingRate'].resample('1D').sum().dropna()
    df_daily.name = 'daily_funding_rate'
    df_daily = df_daily.to_frame()
    df_daily.to_excel("sol_funding.xlsx")

    return True


get_TVL()
get_funding_rates()


True